
# サブワードの埋め込み

:label: `sec_fasttext`

英語では、「helps」、「helped」、「helping」などの単語は、同じ単語「help」の活用形です。 「犬」と「犬」の関係は「猫」と「猫」の関係と同じであり、「男の子」と「彼氏」の関係は「女の子」と「彼女」の関係と同じです。フランス語やスペイン語などの他の言語では、多くの動詞に 40 を超える活用形がありますが、フィンランド語では、名詞に最大 15 の格が存在します。言語学では、形態学は単語の形成と単語の関係を研究します。ただし、単語の内部構造は word2vec でも GloVe でも調査されていません。

##  fastText モデル

word2vec で単語がどのように表現されるかを思い出してください。スキップグラム モデルと連続バッグオブワード モデルの両方で、同じ単語の異なる語形変化は、共有パラメータなしで異なるベクトルによって直接表現されます。形態学的情報を使用するために、 *fastText*モデルは、サブワードが文字 $n$-gram :cite: `Bojanowski.Grave.Joulin.ea.2017`である*サブワード埋め込み*アプローチを提案しました。 fastText は、単語レベルのベクトル表現を学習する代わりに、サブワード レベルのスキップグラムとして考えることができ、各*中心単語は*そのサブワード ベクトルの合計で表されます。

 「where」という単語を使用して、fastText の各中心単語のサブワードを取得する方法を説明します。まず、特殊文字「&amp;lt;」を追加します。そして「&gt;」接頭辞と接尾辞を他のサブワードと区別するために、単語の先頭と末尾に付けられます。次に、単語から文字 $n$-gram を抽出します。たとえば、$n=3$ の場合、長さ 3 のすべてのサブワード「&amp;lt;wh」、「whe」、「her」、「ere」、「re&amp;gt;」、および特別なサブワード「&amp;lt;where&amp;gt;」を取得します。 」。

 fastText では、任意の単語 $w$ について、長さが 3 から 6 までのすべてのサブワードとその特別なサブワードの和集合を $\mathcal{G}_w$ で表します。語彙はすべての単語のサブワードを組み合わせたものです。 $\mathbf{z}_g$ を辞書内のサブワード $g$ のベクトルとすると、スキップグラム モデルの中心単語としての単語 $w$ のベクトル $\mathbf{v}_w$ は次の和になります。そのサブワードベクトル:

 $$\mathbf{v} *w = \sum* {g\in\mathcal{G}_w} \mathbf{z}_g.$$

 fastText の残りの部分はスキップグラム モデルと同じです。スキップグラム モデルと比較して、fastText の語彙はより多く、その結果、より多くのモデル パラメーターが生成されます。さらに、単語の表現を計算するには、そのすべてのサブワード ベクトルを合計する必要があるため、計算がより複雑になります。ただし、類似した構造を持つ単語間でサブワードのパラメーターが共有されるため、まれな単語や語彙外の単語であっても、fastText でより適切なベクトル表現を取得できる可能性があります。

## バイトペアエンコーディング

:label: `subsec_Byte_Pair_Encoding`

 fastText では、抽出されたすべてのサブワードが $3$ から $6$ などの指定された長さである必要があるため、語彙のサイズを事前に定義することはできません。固定サイズの語彙で可変長のサブワードを許可するには、*バイト ペア エンコーディング*(BPE) と呼ばれる圧縮アルゴリズムを適用してサブワードを抽出できます (引用: `Sennrich.Haddow.Birch.2015` )。

バイト ペア エンコーディングは、トレーニング データセットの統計分析を実行して、任意の長さの連続文字など、単語内の共通の記号を検出します。長さ 1 のシンボルから開始して、バイト ペア エンコードでは、最も頻繁に使用される連続シンボルのペアを繰り返しマージして、新しい長いシンボルを生成します。効率を高めるため、単語の境界をまたぐペアは考慮されないことに注意してください。最終的には、このような記号をサブワードとして使用して単語を分割することができます。バイト ペア エンコーディングとそのバリアントは、GPT-2 :cite: `Radford.Wu.Child.ea.2019`や RoBERTa :cite: `Liu.Ott.Goyal.ea.2019`などの一般的な自然言語処理事前トレーニング モデルの入力表現に使用されています。 。以下では、バイト ペア エンコーディングがどのように機能するかを説明します。

まず、記号の語彙をすべての英語の小文字、特殊な語尾記号`&#39;_&#39;` 、および特殊な不明記号`&#39;[UNK]&#39;`として初期化します。


In [1]:
import collections

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']


単語の境界を越えるシンボルのペアは考慮しないため、データセット内の単語をその頻度 (出現回数) にマッピングする辞書`raw_token_freqs`のみが必要になります。特殊記号`&#39;_&#39;`が各単語に追加されているため、出力記号のシーケンス (例: 「a_背の高い人」) から単語シーケンス (例: 「背の高い人」) を簡単に復元できることに注意してください。単一の文字と特殊記号のみの語彙から結合プロセスを開始するため、各単語 (辞書`token_freqs`のキー) 内の連続する文字の各ペアの間にスペースが挿入されます。言い換えれば、スペースは単語内の記号間の区切り文字です。


In [2]:
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}


次の`get_max_freq_pair`関数を定義します。この関数は、単語内の連続するシンボルの最も頻繁に出現するペアを返します。ここで、単語は入力辞書`token_freqs`のキーから取得されます。


In [3]:
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # Key of `pairs` is a tuple of two consecutive symbols
            pairs[symbols[i], symbols[i + 1]] += freq
    return max(pairs, key=pairs.get)  # Key of `pairs` with the max value


連続シンボルの頻度に基づく貪欲なアプローチとして、バイト ペア エンコードでは次の`merge_symbols`関数を使用して、最も頻繁に使用される連続シンボルのペアをマージして新しいシンボルを生成します。


In [4]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs


ここで、辞書`token_freqs`のキーに対してバイト ペア エンコード アルゴリズムを繰り返し実行します。最初の反復では、最も頻繁に出現する連続シンボルのペアは`&#39;t&#39;`と`&#39;a&#39;`であるため、バイト ペア エンコードによってこれらがマージされ、新しいシンボル`&#39;ta&#39;`が生成されます。 2 回目の反復では、バイト ペア エンコーディングにより`&#39;ta&#39;`と`&#39;l&#39;`がマージされ、別の新しいシンボル`&#39;tal&#39;`が生成されます。


In [5]:
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'merge #{i + 1}:', max_freq_pair)

merge #1: ('t', 'a')
merge #2: ('ta', 'l')
merge #3: ('tal', 'l')
merge #4: ('f', 'a')
merge #5: ('fa', 's')
merge #6: ('fas', 't')
merge #7: ('e', 'r')
merge #8: ('er', '_')
merge #9: ('tall', '_')
merge #10: ('fast', '_')



バイト ペア エンコードを 10 回繰り返した後、リスト`symbols`には、他のシンボルから繰り返しマージされたさらに 10 個のシンボルが含まれていることがわかります。


In [6]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']



辞書`raw_token_freqs`のキーで指定された同じデータセットについて、データセット内の各単語は、バイト ペアの結果として、サブワード「fast_」、「fast」、「er_」、「tall_」、「tall」によってセグメント化されるようになりました。エンコードアルゴリズム。たとえば、単語「faster_」と「taller_」は、それぞれ「faster_」と「tall er_」としてセグメント化されます。


In [7]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']



バイト ペア エンコードの結果は、使用されているデータセットによって異なることに注意してください。あるデータセットから学習したサブワードを使用して、別のデータセットの単語をセグメント化することもできます。貪欲なアプローチとして、次の`segment_BPE`関数は、入力引数の`symbols`から単語を可能な限り長いサブワードに分割しようとします。


In [8]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # Segment token with the longest possible subwords from symbols
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs


以下では、前述のデータセットから学習したリスト`symbols`のサブワードを使用して、別のデータセットを表す`tokens`セグメント化します。


In [9]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, symbols))

['tall e s t _', 'fa t t er_']



## まとめ
- fastText モデルは、サブワード埋め込みアプローチを提案します。 word2vec のスキップグラム モデルに基づいて、中心単語をそのサブワード ベクトルの合計として表します。
- バイト ペア エンコードでは、トレーニング データセットの統計分析を実行して、単語内の共通のシンボルを検出します。貪欲なアプローチとして、バイト ペア エンコーディングは、最も頻繁に使用される連続シンボルのペアを繰り返しマージします。
- サブワードの埋め込みにより、珍しい単語や辞書に載っていない単語の表現の品質が向上する可能性があります。

## 演習
1. 例として、英語には約 $3\times 10^8$ の $6$-gram が存在します。サブワードが多すぎると何が問題になるのでしょうか?この問題にどう対処すればよいでしょうか?ヒント: fastText 論文のセクション 3.2 の最後を参照してください:cite: `Bojanowski.Grave.Joulin.ea.2017` 。
1. 連続バッグオブワードモデルに基づいてサブワード埋め込みモデルを設計するにはどうすればよいですか?
1. サイズ $m$ の語彙を取得するには、初期シンボル語彙サイズが $n$ の場合、何回のマージ操作が必要ですか?
1. バイト ペア エンコーディングのアイデアを拡張してフレーズを抽出するにはどうすればよいでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/4587)
